# Feature extraction and transfer learning with pre-trained CNNs

## Task 1: Download training images

Our starting point is to find some pictures of objects that we want our neural network to recognize. To download training images of cats, for instance, you go to https://images.google.com/ and you type in "cat" and you just scroll through until you find a goodly bunch of them (say 200-400)

The next thing you need to do is to get a list of all the URLs there. To do that, you need to open the [developer console](https://support.airtable.com/hc/en-us/articles/232313848-How-to-open-the-developer-console#:~:text=To%20open%20the%20developer%20console%20window%20on%20Chrome%2C%20use%20the,then%20select%20%22Developer%20Tools.%22) in your browser and you paste the following into the window that appears:

(To open the console in Google Chrome hit Ctrl-Shift-J in Windows/Linux and Cmd-Opt-J in Mac)

```
urls=Array.from(document.querySelectorAll('.rg_i')).map(el=> el.hasAttribute('data-src')?el.getAttribute('data-src'):el.getAttribute('data-iurl'));
window.open('data:text/csv;charset=utf-8,' + escape(urls.join('\n')));
```



This will download a text file containing all the URLs. For now, just keep the file on your local machine. Rename the file to `<classname>.csv`. So for instance, if you downloaded URLs of cat images, you want to name the file `cat.csv`

Repeat the above process for each of the object categories, you wish to recognize. Pick 3 or more categories to make it more interesting. Below, I picked cat, dog, and horse.

##Task 2: Mount your Google Drive, create data directory, and upload csv files
Save this notebook to your Google Drive by selecting "Save" or "Save a copy in Drive" in the Files menu. If you want to store data permanently, you also need to mount your Google Drive, which can be done as follows:

**Note:** In my browser, the copy-button that displays next to the authorizatin code doesn't work properly. So I copied the code manually.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Create data directory. I prefer to use pathlab: https://docs.python.org/3/library/pathlib.html, but you can also use normal shell commands by prefixing with !

In [ ]:
from pathlib import Path

root = '/content/gdrive/My Drive/' # Don't change this
#root = '/content/' # Alternative solution if mounting your Drive doesn't work
data_dirname = 'data' # Change as you like
p = Path(root + data_dirname)
p.mkdir(exist_ok=True)

Now, let's upload the csv files to Google Colab. Click the ">" fan at the top left below the Colab logo.

**Note**: The graphical user interface/layout of Colab has changed since I created this notebook, so the instructinos below are not 100% up-to-date. You'll figure it out :-)

![alt text](https://media.geeksforgeeks.org/wp-content/uploads/20190430123759/Screenshot-502.png)

This will open a menu where you can see file hierachy if you click Files.

![alt text](https://media.geeksforgeeks.org/wp-content/uploads/20190430124049/Screenshot-516.png)

To upload the csv files, locate the newly created data directory in the file hierarchy (if you used my code directly the path should be `/content/gdrive/My Drive/data`), then drag-and-drop the csv files from your local machine.

It should result in something like this:

In [ ]:
[print(x) for x in p.iterdir() if x.is_file()]

/content/gdrive/My Drive/data/cat.csv
/content/gdrive/My Drive/data/dog.csv
/content/gdrive/My Drive/data/horse.csv
/content/gdrive/My Drive/data/fish.csv


[None, None, None, None]

If you use shell commands, you can verify that the csv files have been uploaded by listing the content of the data directory:

In [ ]:
!ls '/content/gdrive/My Drive/data'

cat.csv  dog.csv  fish.csv  horse.csv


Note that by default the current directory is /content. You can verify this by running the following command:

In [ ]:
!pwd

/content


##Task 3: Download image files from URLs

The next step is to actually download the image files from the URLs in the csv files.

First specify which classes you want to include. Note that the class names must match the names given to the csv files:

**Note**: Class names must appear in alpha-numeric order to be compatible with the class assignment of the image generator below.

In [ ]:
classes = ['cat','dog','horse']

Then download the images (just ignore any error messages):

In [ ]:
from fastai.vision.data import download_images
from fastai.vision.data import verify_images
max_pics = 400

for idx, name in enumerate(classes):
  print(name)
  folder = name
  file = name + '.csv'
  dest = p/folder
  dest.mkdir(parents=True, exist_ok=True)
  download_images(p/file, dest, max_pics=max_pics)

And remove files that are not actually images (again, just ignore any error messages):

In [ ]:
for c in classes:
    print(c)
    verify_images(p/c, delete=True, max_size=500)

## Task 4: Set up neural network for feature extraction
Before proceeding *REMEMBER TO ENABLE GPU IN THE RUNTIME ENVIRONMENT:* Go to Runtime -> "Change runtime type" and select GPU as hardware acelerator.

We will be using a deep learning framework, called [Keras](https://keras.io/). Keras is a high-level neural network API, written in Python and capable of running on top of [TensorFlow](https://www.tensorflow.org/), CNTK, and Theano.

A common and highly effective approach to deep learning on small image datasets is to leverage a pre-trained network. A pre-trained network is simply a saved network previously trained on a large dataset, such as the [ImageNet dataset](http://www.image-net.org/) (1.4 million labeled images and 1000 different classes). If this original dataset is large enough and general enough, then the spatial feature hierarchy learned by the pre-trained network can effectively act as a generic model of our visual world, and hence its features can prove useful for many different computer vision problems, even though these new problems might involve completely different classes from those of the original task.

In our case, we will consider a convolutional neural network (CNN) trained on ImageNet. We will use the MobileNet architecture, but there are other models that you could use as well. Take a look here: https://keras.io/applications

There are two ways to leverage a pre-trained network: feature extraction and fine-tuning. We will be covering both of them today. Let's start with feature extraction.

Feature extraction consists of using the representations learned by an existing neural network to extract interesting features from new samples. These features are then run through a new classifier, which is trained from scratch. This could be any classifier, such as K-Nearest Neighbours (K-NN).

Traditional CNNs are divided into two parts: they start with a series of convolution and pooling layers, and they end with a densely-connected classifier. The first part is often referred to as the "encoder", "feature extractor" or "convolutional base" of the model. In the case of CNNs, "feature extraction" will simply consist of taking the convolutional base of a previously-trained network, running the new data through it, and training a new classifier on top of the output. The second part of the network, called the "decoder" or "top layers", is ignored for now. We will be using it for Transfer Learning in Task 11.

First, let's download and instantiate the pre-trained MobileNet without the top layers (i.e., without the decoder):

###Caution
For some reason, you may sometimes need to downgrade tensorflow to use keras. To do this, run the following command:

In [ ]:
#!pip install --upgrade tensorflow==1.8.0

In [ ]:
from keras.applications import MobileNet

conv_base = MobileNet(weights='imagenet',
                      include_top=False,
                      input_shape=(120, 120, 3))

Let's summarize the model used for feature extraction:

In [ ]:
conv_base.summary()

###Questions 4.1
1. What is the expected shape of the input image?
2. What is the shape of the output of the model?
3. What happens to the output shape if you double the size of the input image?
4. Can you guess what the None dimension is used for?

##Task 5: Extract features from an image
Many neural networks expect the input image to have a fixed, pre-defined shape. Also, the pixel intensities are assumed to be in a fixed range. So for instance, if you train a network on images with intensities in the range -127.5 to 127.5, and you then feed the same network images with intensities in the range -0.5 to 0.5, the output of the network will most likely be garbage.

Fortunately each pre-trained network in Keras comes with its own *preprocessor*, which assures that the intensities are scaled correctly for that particular network.

Let's load an image, preprocess it, and feed it through the network:

In [ ]:
import numpy as np
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input

# Pick first image of first class
filelist = [x for x in (p/classes[0]).iterdir() if x.is_file()]
img_path = filelist[1]
print(f"File path: {img_path}")

# Load image and preprocess it
img = image.load_img(img_path, target_size=(120, 120))
img_data = image.img_to_array(img)
img_data = np.expand_dims(img_data, axis=0)
img_preprocessed = preprocess_input(img_data.copy())

# Feed preprocessed image through CNN encoder to get a new feature representation
mobilenet_features = conv_base.predict(img_preprocessed)

File path: /content/gdrive/My Drive/data/cat/00000001.jpg


###Questions 5.1
1. What is the range of the pixel values before and after preprocessing?
2. So what formula do you think is used to pre-process the pixel values?
3. What is the order of the color channels? (you could compare with openCV)?
4.a) What is the size of the input image?
4.b) What is the size of the calculated feature representation (`mobilenet_features`)?
4.c) So what is the reduction in dimensionality after feature extraction?

###Extra - exploring the model further
We can explore the model even further. Let's look at the convolution layers:

In [ ]:
for i, layer in enumerate(conv_base.layers):
  
  # check for convolutional layer
  layer_type = layer.__class__.__name__
  
  if 'Conv' not in layer_type:
    continue
  
  # get filter weights
  layer_name = layer.name
  input_shape = layer.input_shape
  output_shape = layer.output.shape
  filter_shape = layer.get_weights()[0].shape
  
  print(f"Layer {i} has name {layer_name}, input shape {input_shape}, filter shape {filter_shape}, and output shape {output_shape}")

Display some filters from layer 2:

In [ ]:
import matplotlib.pyplot as plt
layer = conv_base.layers[2] # Pick layer 2 from the list above
filters = layer.get_weights()
plt.figure(figsize=(12,12))
for i in range(32):
  f = filters[0][:,:,:,i]
  
  # Normalize to range 0 ... 1
  f -= f.min()
  f /= f.max()
  
  plt.subplot(4,8,i+1)
  plt.imshow(f)
  plt.title(str(i))

And display the resulting outputs (called **feature maps**):

In [ ]:
from keras import Model
dummy_model = Model(inputs=conv_base.input, outputs=conv_base.get_layer('conv1').output) # See conv_base.summary() for complete list of layer names
out = dummy_model.predict(img_preprocessed)
out = np.reshape(out,(60,60,32))

plt.figure(figsize=(16,16))
for i in range(32):
  f = out[:,:,i]
  plt.subplot(4,8,i+1)
  plt.imshow(f,cmap='gray')
  plt.title("{0:.2f}".format(f.min()) + "/" + "{0:.2f}".format(f.max()))

Layers you could also inspect (layer names are written in **bold**):

**conv1_pad** (ZeroPadding2D)    (None, 121, 121, 3)       0         
_________________________________________________________________
**conv1** (Conv2D)               (None, 60, 60, 32)        864       
_________________________________________________________________
**conv1_bn** (BatchNormalization (None, 60, 60, 32)        128       
_________________________________________________________________
**conv1_relu** (ReLU)            (None, 60, 60, 32)        0         
_________________________________________________________________
**conv_dw_1** (DepthwiseConv2D)  (None, 60, 60, 32)        288       
_________________________________________________________________
**conv_dw_1_bn** (BatchNormaliza (None, 60, 60, 32)        128       
_________________________________________________________________
**conv_dw_1_relu** (ReLU)        (None, 60, 60, 32)        0         
_________________________________________________________________
**conv_pw_1** (Conv2D)           (None, 60, 60, 64)        2048      
_________________________________________________________________
**conv_pw_1_bn** (BatchNormaliza (None, 60, 60, 64)        256       
_________________________________________________________________
**conv_pw_1_relu**(ReLU)        (None, 60, 60, 64)        0         
_________________________________________________________________
**conv_pad_2** (ZeroPadding2D)   (None, 61, 61, 64)        0         

You may find more knowledge about the layer types here:
- https://towardsdatascience.com/a-basic-introduction-to-separable-convolutions-b99ec3102728
- https://towardsdatascience.com/batch-normalization-in-neural-networks-1ac91516821c
- https://en.wikipedia.org/wiki/Rectifier_(neural_networks)

##Task 6: How to use the image generator 
Loading and preprocessing images is such a common thing in deep learning that frameworks like Keras provide predefined tools for us that we can use. 
In this task we will look at Keras' image data generator: https://keras.io/preprocessing/image/#imagedatagenerator-class. Simply put, the image generator is a tool that makes loading and preprocessing data easy.

Let's set up an image generator that outputs mini-batches of 32 images:


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies
generator = datagen.flow_from_directory(str(p), # this is where you specify the path to the main data folder
                                        target_size=(120,120),
                                        color_mode='rgb',
                                        batch_size=32,
                                        class_mode='categorical',
                                        shuffle=True)

**Note:** Check that the classes assigned by the generator are consistent with your class assignment:

In [ ]:
print(generator.class_indices)
print(classes)

{'cat': 0, 'dog': 1, 'horse': 2}
['cat', 'dog', 'horse']


Here is one way to generate a new batch:

In [ ]:
inputs, labels = generator.next()

###Questions 6.1
1. What is variable "inputs"?
2. What is variable "labels"?
3. How does the image generator know where the images are stored?
4. How does the image generator know the class of each image?
5. What does shuffle mean?

As you learned in lecture 2 it is always a good idea to split the data into a training set and a validation set. Again, this is such a common thing in deep learning that the image generator can do it for us:

In [ ]:
datagen = ImageDataGenerator(preprocessing_function=preprocess_input,validation_split=0.2)

train_generator = datagen.flow_from_directory(str(p),
                                        target_size=(120,120),
                                        color_mode='rgb',
                                        batch_size=32,
                                        class_mode='categorical',
                                        shuffle=True,
                                        subset='training')
validation_generator = datagen.flow_from_directory(str(p),
                                        target_size=(120,120),
                                        color_mode='rgb',
                                        batch_size=32,
                                        class_mode='categorical',
                                        shuffle=True,
                                        subset='validation')

###Questions 6.2
1. How does each of the two generators know if it should produce training or validation images?
2. What is the validation percentage in this example?

Now, you can in principle create a complete training set and a validation set of images:

In [ ]:
def extract_features(generator,batch_size,num_batches):
    sample_count = batch_size * num_batches
    features = np.zeros(shape=(sample_count, 120*120*3))
    labels = np.zeros(shape=(sample_count))
    i = 0
    for inputs_batch, labels_batch in generator:
        print(i)
        # Use np.reshape to convert images into vectors
        features_batch = np.reshape(inputs_batch,(32,120*120*3))
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = np.argmax(labels_batch,axis=1)
        i += 1
        if i * batch_size >= sample_count:
            # Note that since generators yield data indefinitely in a loop,
            # we must `break` after every image has been seen once.
            break
    return features, labels

num_train_batches = 20
num_validation_batches = 4 
train_features_raw, train_labels_raw = extract_features(train_generator, 32, num_train_batches)
validation_features_raw, validation_labels_raw = extract_features(validation_generator, 32, num_validation_batches)

Show the first 32 images of the validation data set (verify that the labels are correct):

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(9,9))
for i in range(32):
  img = (np.reshape(validation_features_raw[i,:],(120,120,3))+1)/2
  plt.subplot(4,8,i+1)
  plt.imshow(img)
  plt.xticks([]), plt.yticks([])
  plt.title(classes[int(validation_labels_raw[i])])

###Questions 6.3
The image data has now been vectorized.
1. What is the shape of ```train_features_raw``` and ```train_labels_raw```?
2. What is the shape of ```validation_features_raw``` and ```validation_labels_raw```?
3. Notice the values of ```num_train_batches``` and ```num_validation_batches```. What are the implications of this?

##Task 7: Classify images using K-NN and raw pixels
The features that we have just calculated correspond to the raw pixel values. Now, your task is to train a K-NN classifier on the training set, and evaluate the performace on the validation set (i.e., what is the accuracy on the validation set?)

The training set consists of variables

```
train_features_raw, train_labels_raw
```

and the validation set consists of variables

```
validation_features_raw, validation_labels_raw
```


You are on your own here. You don't have to implement K-NN yourself. I suggest you use [scikit-learn](https://scikit-learn.org). Personally, I found this tutorial quite useful: https://scikit-learn.org/stable/auto_examples/neighbors/plot_classification.html#sphx-glr-auto-examples-neighbors-plot-classification-py

##Task 8: Classify images using K-NN and neural net features
Now, repeat the same task, but this time using the features calculated using the pre-trained neural network, rather than the raw pixel values.

(*NOTE: Running this code block sometimes courses an error, when I run it. I haven't found a way to fix it yet. If it happens to you, just keep running the code block until it doesn't fail...*)

In [ ]:
def extract_features(generator,batch_size,num_batches):
    sample_count = batch_size * num_batches
    features = np.zeros(shape=(sample_count, 3*3*1024))
    labels = np.zeros(shape=(sample_count))
    i = 0
    for inputs_batch, labels_batch in generator:
        print(i)
        # This is where we apply the CNN encoder to "convert" the image into a feature vector
        features_batch = conv_base.predict(inputs_batch)
        features_batch = np.reshape(features_batch,(32,3*3*1024))
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = np.argmax(labels_batch,axis=1)
        i += 1
        if i * batch_size >= sample_count:
            # Note that since generators yield data indefinitely in a loop,
            # we must `break` after every image has been seen once.
            break
    return features, labels

num_train_batches = 20
num_validation_batches = 4
train_features, train_labels = extract_features(train_generator, 32, num_train_batches)
validation_features, validation_labels = extract_features(validation_generator, 32, num_validation_batches)

If everything went as planned, you should be able to conclude that the accuracy of the K-NN classifier is significantly higher when using the neural networks features compared to when using the raw pixel values as features. This is becuase the MobileNet has already been pre-trained, i.e., it has learned features that are useful for classifying 1000 different object categories. In almost any scenario you can think of, MobileNet's feature representation will be better than using the raw pixels.

##Task 9: K-means clustering
So what we have learned so far is that images of the same class tend to group closer together when using MobileNet's feature representation, but not so much when using the raw intensities. This confirms that using the raw pixels as features is in general a bad idea.

The reason that MobileNet's feature representation works better is because the network has learned to map images onto a manifold. A manifold is kind of like a low-dimensional surface that exists in a high-dimensional space. For instance if images of faces were to be mapped into a 4D manifold, the first axis on the manifold could represent gender, and the others could represent age, view angle, and eye color. You can read more about manifold learning in chapter 5.11.3 of [the book](https://github.com/janishar/mit-deep-learning-book-pdf).

The underlying hypothesis of using K-NN to classifiy images based on the features computed by MobileNet is that *objects that are similar will map to the approximate same location on some manifold.* Here we will perform K-means clustering and verify that this is in fact the case. For the record, recall that the K-means method is an *undersupervised learning method*, so it doesn't know anything about the class labels.

Your task is to perform K-means clustering twice on your dataset: first using the raw intensity features (```train_features_raw```), then using the MobileNet features (```train_features```). Use as many clusters as you have classes.

Again, you don't have to implement K-menas clsutering from scratch. You can use scikit-learn. I used this tutorial for inspiration: https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_digits.html#sphx-glr-auto-examples-cluster-plot-kmeans-digits-py

For each cluster, print the class labels of all images in that cluster. Explain what you observe and compare between MobileNet features and raw pixel intensities.

##Task 10: Image retrieval
You can use the features of MobileNet to implement an image search engine. This is also called image retrieval.

To make this work you need to create a new database, where each entry contains an image **and** its feature vector (as computed by MobileNet).

The search engine work like this:

1. Given an input image, pre-process it and feed it through MobileNet to calculate the feature vector.
2. Then perform a K-NN search with K=10 against the feature vectors in the database.
3. Then return the corresponding 10 closest images (also stored in the database).

##Task 11: Transfer learning


Putting your own K-NN classifier on top of a pre-trained CNN is not really optimal. Why? Because, while the features of the convolutional base are better than using raw pixel values, they are not guaranteed to 100% optimal for your specific task. So, a better solution is to attach a second neural network on top of the convolutional base, and train both the classifier *and* the convolutional base at the same time. This is called **transfer learning**. The extra neural network that put on top of our encoder is often referred to as a "decoder".

Recall that CNNs like AlexNet and MobileNet have been trained on ImageNet, which contains 1000 classes. If you download Keras' pre-trained models *including the top layers* (i.e., the decoder), the top layers are in fact the classifier that we want to replace. Let's verify this:

In [ ]:
mobilenet_full = MobileNet(weights='imagenet',
                      include_top=True,
                      input_shape=(224, 224, 3))
mobilenet_full.summary()

###Questions 11.1
Inspect the printout above.
1.  Can you identify the convolutional base of this network? (Compare to the ```conv_base``` model we used earlier.)
2. All layers beyond the convolutional base represent the classifier (or decoder). How many classes are there?
3. So what is the size of the output of the model?
4. Can you guess how we should interpret the output of model?
5. The input size must be 224 by 224 pixels (you can verify for yourself). Why do you think that is? 

So, how do we modify and re-train MobileNet to work on your own data? First of all, we don't want to train CNNs from scratch, since this could take days. Secondly, we need to modify the network architecture to output, say, three class labels instead of 1000.

The main hypothesis underlying transfer learning is that the network weights learned in the convolutional layers (i.e., the *encoder*) are generic and need little or no fine-tuning to work on other data sets or tasks. So in practice, we just need to replace and re-train the last layers (i.e., the *decoder*) of a pre-trained network.

So let's take our convolutional base (encoder) and put a simple neural network classifier (decoder) on top of it. Your task is to figure out what the value of variable N should be.

In [ ]:
N = ???

In [ ]:
from keras.layers import Dense,GlobalAveragePooling2D
from keras.models import Model

# Add new top layer
x = conv_base.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024,activation='relu')(x) #dense layer
preds = Dense(N,activation='softmax')(x) #final layer with softmax activation

# Specify model
model = Model(inputs=conv_base.input, outputs=preds)

Note that the weights of the new dense layers are initialized with random values.

###Questions 11.2
1. What should N be in the above code block?
2. Re-run the code block with the correct N.
3. What does GlobalAveragePooling2D do?

Hint: You can print all layers and print properties like name, type and input shape:

We will only be training the new dense layers that we added. Disable training for all previous layers and enable for new layers:

In [ ]:
total_num_layers = len(model.layers)
num_base_layers = len(conv_base.layers)
print(f"Total number of layers is {total_num_layers}")
print(f"Number of pretrained base layers is {num_base_layers}")

for layer in model.layers[:num_base_layers]:
    layer.trainable=False
for layer in model.layers[num_base_layers:]:
    layer.trainable=True

Total number of layers is 90
Number of pretrained base layers is 87


We are now ready to start training the model using
- Adam optimizer
- loss function will be categorical cross entropy
- evaluation metric will be accuracy


In [ ]:
from keras import optimizers

# Set up optimizer
sgd_optimizer = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)

# Compile model - make it trainable
model.compile(optimizer=sgd_optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

step_size_train = train_generator.n//train_generator.batch_size # Number of mini-batches per epoch (training)
step_size_val = validation_generator.n//validation_generator.batch_size # Number of mini-batches per epoch (validation)

# Train model for 10 epochs
history = model.fit_generator(generator=train_generator,
                   validation_data=validation_generator,
                   validation_steps=step_size_val,
                   steps_per_epoch=step_size_train,
                   epochs=10)

###Questions 11.3
Look at the outputs of the training.

1. What is the difference between 'loss' and 'val_loss'?
2. What is the difference between 'accuracy' and 'val_accuracy'?
3. Do they behave the same, or do they behave differently? Try to explain what you see.

##Ideas for further work
1. In the above example we have not optimized the pre-trained weights of the convolutional base (i.e., the encoder). To improve performance further you could enable training in all layers (including the convolutoinal base) and re-train the network. This is called *fine-tuning*.
2. Another way to improve model performance is by *data augmentation*. Have a look at the documentation for the [image generator class](https://keras.io/preprocessing/image/) and see what kind of augmentation is possible. Why do you think data augmentation helps improve the performance of your model?
3. Try repeating the above experiments on the MNIST dataset. Take a look here: https://keras.io/examples/mnist_cnn/

###Hints:
1. To enable training of all layers:

In [ ]:
for layer in model.layers[:num_base_layers]:
    layer.trainable=True

2. See examples of data augmentation here:

https://machinelearningmastery.com/how-to-configure-image-data-augmentation-when-training-deep-learning-neural-networks/

In [ ]:
import cv2

# Pick first image of first class
filelist = [x for x in (p/classes[0]).iterdir() if x.is_file()]
img_path = str(filelist[1])
print(f"File path: {img_path}")

# Pick first image of first class
filelist = [x for x in (p/classes[0]).iterdir() if x.is_file()]
img_path = filelist[1]
print(f"File path: {img_path}")

img = image.load_img(img_path, target_size=(120, 120))
img_data = image.img_to_array(img)
img_data = np.expand_dims(img_data, axis=0)

# create image data augmentation generator
datagen = ImageDataGenerator(width_shift_range=[-20,20],
                             height_shift_range=0.5,
                             horizontal_flip=True,
                             rotation_range=30,
                             brightness_range=[0.2,1.0])
# prepare iterator
it = datagen.flow(img_data, batch_size=1)

plt.figure(figsize=(9,9))
# generate samples and plot
for i in range(9):
	# define subplot
	plt.subplot(330 + 1 + i)
	# generate batch of images
	batch = it.next()
	# convert to unsigned integers for viewing
	img = batch[0].astype('uint8')
	# plot raw pixel data
	plt.imshow(img)
# show the figure
plt.show()

3. See below how to load MNIST. Your main challenge is going to be that that input images are 28x28x1, and MobileNt accepts only images that are 32x32x3 or larger. So what to do?

In [ ]:
import keras
from keras.datasets import mnist

num_classes = 10

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

# Convert to float32
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# This will produce an error...
model.fit(x_train, y_train,
          batch_size=32,
          epochs=100,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

##Optional task: Deploying the model
Here is how to deploy the model and integrate with OpenCV.

In [ ]:
import cv2

# Pick first image of first class
filelist = [x for x in (p/classes[0]).iterdir() if x.is_file()]
img_path = str(filelist[1])
print(f"File path: {img_path}")

# Load and display with OpenCV (rememebr to convert to RGB!!!)
img = cv2.imread(img_path)
b,g,r = cv2.split(img)
img = cv2.merge((r,g,b))
plt.imshow(img)

Now, make sure that the image shape and the pixel intensity range is as expected by the network:

In [ ]:
img = cv2.resize(img, (120, 120))
img = (img[...,::-1].astype(np.float32))
img /= 127.5
img -= 1.
img = np.expand_dims(img,0)
print(img.shape)

Run the image through the network:

In [ ]:
pred = model.predict(img)[0]
ind = (-pred).argsort()[:3]
latex = [(classes[x],pred[x]) for x in ind]
print(latex)

In [ ]:
print(pred)

##Optional task: Exporting to TensorFlow JS and hosting a web service on GitHub
If you want to, you can deploy your model and make a nice web service like this one:
https://klaverhenrik.github.io/transferlearning/

To do that, first export your model to [TensorFlow JS](https://www.tensorflow.org/js) and download all the necessary files as a zip file:

In [ ]:
!mkdir model
!tensorflowjs_converter --input_format keras keras.h5 model/

/bin/bash: tensorflowjs_converter: command not found


In [ ]:
with open('class_names.txt', 'w') as file_handler:
    for item in classes:
        file_handler.write("{}\n".format(item))

In [ ]:
with open('my_classes.js','w') as file_handler:
  file_handler.write("export const IMAGENET_CLASSES = {\n")
  for ix, item in enumerate(classes):
    file_handler.write("  " + str(ix) + ": \'" + item + "\'")
    if ix < len(classes)-1:
      file_handler.write(",")
    file_handler.write("\n")
  file_handler.write("};")

In [ ]:
!pip install tensorflowjs 

In [ ]:
model.save('keras.h5')

In [ ]:
!mkdir model
!tensorflowjs_converter --input_format keras keras.h5 model/

In [ ]:
!cp class_names.txt model/class_names.txt
!cp my_classes.js model/my_classes.js

In [ ]:
!zip -r model.zip model 

In [ ]:
from google.colab import files
files.download('model.zip')

Now that you have converted you model to TensorFlow JS and downloaded, you can clone [this git repo](https://github.com/klaverhenrik/klaverhenrik.github.io/tree/master/transferlearning) and copy the model files into your own copy of the repo.

The webpage can be hosted on GitHub using [GitHub Pages](https://pages.github.com/).